In [1]:
import os

os.environ["BRASA_DATA_PATH"] = "D:\\brasa"

In [2]:
import sys
sys.path.append('..')

from datetime import datetime

import pandas as pd
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pyarrow
from bizdays import Calendar, set_option

import brasa

man = brasa.CacheManager()

In [3]:
ds_intra = brasa.get_dataset("b3-trades-intraday")

In [4]:
ds_intra.count_rows()

187462030

In [5]:
ds_intra.schema

refdate: timestamp[us]
symbol: string
update_action: int64
traded_price: double
traded_quantity: int64
trade_time: string
trade_id: int64
trade_session_id: int64
trade_date: timestamp[us]
buyer_code: int64
seller_code: int64
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1673

In [6]:
ds_intra.filter(pc.field("symbol") == "DI1F24").count_rows()

202607

In [8]:
tb_di1f24 = ds_intra.filter(pc.field("symbol") == "DI1F24").to_table()

In [10]:
df_di1f24 = tb_di1f24.to_pandas()

In [11]:
df_di1f24

,refdate,symbol,update_action,traded_price,traded_quantity,trade_time,trade_id,trade_session_id,trade_date,buyer_code,seller_code
0,2023-05-31,DI1F24,0,13.210,10,090048610,10,1,2023-05-31,114,114
1,2023-05-31,DI1F24,0,13.210,27,090048610,20,1,2023-05-31,1618,114
2,2023-05-31,DI1F24,0,13.205,4,090054707,30,1,2023-05-31,1618,1618
3,2023-05-31,DI1F24,0,13.205,6,090100660,40,1,2023-05-31,1618,1618
4,2023-05-31,DI1F24,0,13.205,6,090100697,50,1,2023-05-31,1618,8
...,...,...,...,...,...,...,...,...,...,...,...
202602,2023-06-28,DI1F24,0,12.970,500,175932270,54410,1,2023-06-28,3,3
202603,2023-06-28,DI1F24,0,12.970,125,175954357,54420,1,2023-06-28,72,72
202604,2023-06-28,DI1F24,0,12.970,32,175956920,54430,1,2023-06-28,72,72
202605,2023-06-28,DI1F24,0,12.970,9,175957587,54440,1,2023-06-28,72,72
